In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_predict, KFold
from sklearn.model_selection import train_test_split

In [3]:
# data = pd.read_excel('path')
data = pd.read_excel('GFA_Dataset.xlsx')

In [4]:
X = data.drop('label', axis=1)
y_true = data['label']
X_train,X_test,y_train,y_test = train_test_split(X,y_true,test_size=0.10)

In [5]:
model = xgb.XGBClassifier(learning_rate = 0.4, n_estimators = 500, max_depth = 4)
params = model.get_params()
for param, value in params.items():
    print(f"{param}: {value}")

objective: binary:logistic
use_label_encoder: None
base_score: None
booster: None
callbacks: None
colsample_bylevel: None
colsample_bynode: None
colsample_bytree: None
early_stopping_rounds: None
enable_categorical: False
eval_metric: None
feature_types: None
gamma: None
gpu_id: None
grow_policy: None
importance_type: None
interaction_constraints: None
learning_rate: 0.4
max_bin: None
max_cat_threshold: None
max_cat_to_onehot: None
max_delta_step: None
max_depth: 4
max_leaves: None
min_child_weight: None
missing: nan
monotone_constraints: None
n_estimators: 500
n_jobs: None
num_parallel_tree: None
predictor: None
random_state: None
reg_alpha: None
reg_lambda: None
sampling_method: None
scale_pos_weight: None
subsample: None
tree_method: None
validate_parameters: None
verbosity: None


In [6]:
kf = KFold(n_splits=10, shuffle=True)
confusion_matrices = []
classification_reports = []

In [7]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y_true.iloc[train_index], y_true.iloc[test_index]

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    cr = classification_report(y_test, y_pred)

    confusion_matrices.append(cm)
    classification_reports.append(cr)

In [8]:
for i, (cm, cr) in enumerate(zip(confusion_matrices, classification_reports)):
    print(f"Fold {i+1}:")
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(cr)
    print("-" * 50)

Fold 1:
Confusion Matrix:
[[ 71  27]
 [ 16 582]]
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.72      0.77        98
           1       0.96      0.97      0.96       598

    accuracy                           0.94       696
   macro avg       0.89      0.85      0.87       696
weighted avg       0.94      0.94      0.94       696

--------------------------------------------------
Fold 2:
Confusion Matrix:
[[ 81  23]
 [ 15 577]]
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.78      0.81       104
           1       0.96      0.97      0.97       592

    accuracy                           0.95       696
   macro avg       0.90      0.88      0.89       696
weighted avg       0.94      0.95      0.94       696

--------------------------------------------------
Fold 3:
Confusion Matrix:
[[ 84  23]
 [ 12 577]]
Classification Report:
              precision    r

In [9]:
average_cm = np.mean(confusion_matrices, axis=0)
y_pred_all = cross_val_predict(model, X, y_true)
average_cr = classification_report(y_true, y_pred_all)
print("Average Confusion Matrix:")
print(average_cm)
print('Accary:', (cm[1,1]+cm[0,0])/(cm[1,1]+cm[0,1]+cm[0,0]+cm[1,0]))

Average Confusion Matrix:
[[ 75.8  29.2]
 [ 15.7 575. ]]
Accary: 0.9280575539568345
